## Analizador Lexico
Un analizador lexico transforma una entrada en tokens

In [54]:
from enum import Enum

class TipoToken(Enum):  #sistema para clasificación de palabras y conectores
    O = 1
    Y = 2
    S = 3
    N = 4
    E = 5
    P = 6

In [55]:
lexemas = []

def Analizador_lexico(entrada : str) -> list:
    lexemas.clear()
    tokens = entrada.lower().split()
    for token in tokens:
        if token == 'o':
            lexemas.append({"valor": token , "tipo": TipoToken.O })
        elif token == 'y':
            lexemas.append({"valor": token , "tipo": TipoToken.Y})
        elif token == 'si':
            lexemas.append({"valor": token , "tipo": TipoToken.S})
        elif token == 'entonces':
            lexemas.append({"valor": token , "tipo": TipoToken.E})
        elif token == 'no':
            lexemas.append({"valor": token , "tipo": TipoToken.N})
        else :
            lexemas.append({"valor":token , "tipo": TipoToken.P })
            
    return(lexemas)

## Analizador Sintactico

In [56]:
class EstadosAnalizador(Enum):  #Sistema de guardado de Estados
    INICIO = 1
    PRECEDENTE = 2
    NEGACION = 3
    CONSECUENTE = 4
    DISYUNCION = 5
    CONJUNCION = 6
    ATOMO = 7
    ERROR1 = 8  #se esperaba un 'si', 'no' o un atomo
    ERROR2 = 9  #se esperaba un 'no' o un atomo
    ERROR3 = 10 #se esperaba un 'y', 'o', 'entonces' o un atomo
    ERROR4 = 11 #se esperaba un atomo
    ERROR5 = 12 #no se encontro un 'si' antes de 'entonces'
    FIN = 13

In [57]:
l_reglas = []
atomos = []

def Analizador_sintactico(lexemas : list):
    regla = ""
    pila = ""
    prec = 0
    pila = []
    prop = ""

    EstadoActual = EstadosAnalizador.INICIO

#INICIO
    for lexema in lexemas:
        if EstadoActual == EstadosAnalizador.INICIO:
            if lexema['valor'] == "si":
                EstadoActual = EstadosAnalizador.PRECEDENTE
                prec = 1
            elif lexema['valor'] == "no":
                EstadoActual = EstadosAnalizador.NEGACION
                if regla:
                    regla += " "
                regla += "~"
            elif lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            else:
                EstadoActual = EstadosAnalizador.ERROR1

#Precedente
        elif EstadoActual == EstadosAnalizador.PRECEDENTE:
            if lexema['valor'] == "no":
                EstadoActual = EstadosAnalizador.NEGACION
                if regla:
                    regla += " "
                regla += "~"
            elif lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            else:
                EstadoActual = EstadosAnalizador.ERROR2

#Negacion
        elif EstadoActual == EstadosAnalizador.NEGACION:
            if lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            else:
                EstadoActual = EstadosAnalizador.ERROR3

#Consecuente
        elif EstadoActual == EstadosAnalizador.CONSECUENTE:
            if lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            elif lexema['valor'] == "no":
                EstadoActual = EstadosAnalizador.NEGACION
                if regla:
                    regla += " "
                regla += "~"
            else:
                EstadoActual = EstadosAnalizador.ERROR2

#Atomo
        elif EstadoActual == EstadosAnalizador.ATOMO:
            if lexema['valor'] == "entonces" and prec == 1:
                EstadoActual = EstadosAnalizador.CONSECUENTE
                prop = " ".join(pila)
                atomos.append(prop)
                if regla:
                    regla += " "
                regla += prop
                if regla:
                    regla += " "
                regla += "->"
                pila.clear()
                prop = ""
            elif lexema['valor'] == "y":
                EstadoActual = EstadosAnalizador.CONJUNCION
                prop = " ".join(pila)
                atomos.append(prop)
                if regla:
                    regla += " "
                regla += prop
                if regla:
                    regla += " "
                regla += "&"
                pila.clear()
                prop = ""
            elif lexema['valor'] == "o":
                EstadoActual = EstadosAnalizador.DISYUNCION
                prop = " ".join(pila)
                atomos.append(prop)
                if regla:
                    regla += " "
                regla += prop
                if regla:
                    regla += " "
                regla += "V"
                pila.clear()
                prop = ""
            elif lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            elif lexema['valor'] == "entonces" and prec == 0:
                EstadoActual = EstadosAnalizador.ERROR5
            else:
                EstadoActual = EstadosAnalizador.ERROR3

#Conjuncion
        elif EstadoActual == EstadosAnalizador.CONJUNCION:
            if lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            elif lexema['valor'] == "no":
                EstadoActual = EstadosAnalizador.NEGACION
                if regla:
                    regla += " "
                regla += "~"
            else:
                EstadoActual = EstadosAnalizador.ERROR2
                
#Disyuncion
        elif EstadoActual == EstadosAnalizador.DISYUNCION:
            if lexema['tipo'] == TipoToken.P:
                EstadoActual = EstadosAnalizador.ATOMO
                pila.append(lexema['valor'])
            elif lexema['valor'] == "no":
                EstadoActual == EstadosAnalizador.NEGACION
                if regla:
                    regla += " "
                regla += "~"
            else:
                EstadoActual = EstadosAnalizador.ERROR2

#Errores
        elif EstadoActual == EstadosAnalizador.ERROR1:
            print("se esperaba un 'si', 'no' o un atomo")
            regla = ""
            prop = ""
            pila.clear()
            break
        elif EstadoActual == EstadosAnalizador.ERROR2:
            print("se esperaba un 'no' o un atomo")
            regla = ""
            prop = ""
            pila.clear()
            break
        elif EstadoActual == EstadosAnalizador.ERROR3:
            regla = ""
            prop = ""
            pila.clear()
            print("se esperaba un 'y', 'o', 'entonces' o un atomo")
            break
        elif EstadoActual == EstadosAnalizador.ERROR4:
            print("se esperaba un atomo")
            regla = ""
            prop = ""
            pila.clear()
            break
        elif EstadoActual == EstadosAnalizador.ERROR5:
            print("no se encontro un 'si' antes de 'entonces'")
            regla = ""
            prop = ""
            pila.clear()
            break
        else:
            EstadoActual == EstadosAnalizador.FIN

#Salida sin errores
    if EstadoActual != (EstadosAnalizador.ERROR1 and EstadosAnalizador.ERROR2 and EstadosAnalizador.ERROR3 and EstadosAnalizador.ERROR4 and EstadosAnalizador.ERROR5):
        prop = " ".join(pila)
        atomos.append(prop)
        if regla and prop:
            regla += " " + prop
        else:
            regla = prop
        l_reglas.append(regla)
        prop = ""
        pila.clear()
        regla = ""
        print("proposicion valida")
    
    return(l_reglas, atomos)

In [62]:
Palabras = []
l_reglas.clear()
lexemas.clear()
atomos.clear()

frases = [
    "Si es de dia o es de noche entonces existo",
    "Es de dia"]

for frase in frases:
    Palabras.clear()
    Palabras = Analizador_lexico(frase)
    Analizador_sintactico(Palabras)

print(l_reglas)
print(atomos)

proposicion valida
proposicion valida
['es de dia V es de noche -> existo', 'es de dia']
['es de dia', 'es de noche', 'existo', 'es de dia']


## Prueba de lectura de archivos

In [73]:
Palabras = []
l_reglas.clear()
lexemas.clear()
atomos.clear()

with open("reglas-texto.txt", "r") as archivo:
    lineas = archivo.readlines()

erease = "\n"
enun = []
for linea in lineas:
    ent = linea.replace(erease, "")
    enun.append(ent)

for frase in enun:
    Palabras.clear()
    Palabras = Analizador_lexico(frase)
    Analizador_sintactico(Palabras)
        

proposicion valida
proposicion valida
proposicion valida
proposicion valida
proposicion valida
proposicion valida
proposicion valida
proposicion valida
proposicion valida
proposicion valida


['tiene orejas puntiagudas',
 'es un elfo oscuro',
 'es un elfo alto',
 'es un elfo del bosque',
 'tiene pelaje',
 'sus ojos tienen pupilas verticales',
 'es un kahjiit',
 'es un argoniano',
 'es un elfo alto',
 'es un elfo oscuro',
 'es un elfo del bosque',
 'tiene piel oscura',
 'es un elfo oscuro',
 'es un guarda rojo',
 'tiene piel clara',
 'tiene orejas puntiagudas',
 'es un nordico',
 'tiene colmillos',
 'es un orco',
 'es un kahjiit',
 'es un ser humano',
 'es un nordico',
 'es un guarda rojo',
 'es un imperial',
 'es un breton',
 'usa magia de forma natural',
 'es un breton',
 'es un elfo oscuro',
 'es un alto elfo',
 'es muy habil con las armas pesadas',
 'es un orco',
 'es un nordico',
 'es un guarda rojo',
 'es muy habil al ser sigiloso',
 'es un kahjiit',
 'es un elfo del bosque']

## Elimina atomos repetidos en la lista de atomos

In [72]:
seen = set()
atomos_unicos = []

for atomo in atomos:
    if atomo not in seen:
        atomos_unicos.append(atomo)
        seen.add(atomo)

atomos_unicos

['tiene orejas puntiagudas',
 'es un elfo oscuro',
 'es un elfo alto',
 'es un elfo del bosque',
 'tiene pelaje',
 'sus ojos tienen pupilas verticales',
 'es un kahjiit',
 'es un argoniano',
 'tiene piel oscura',
 'es un guarda rojo',
 'tiene piel clara',
 'es un nordico',
 'tiene colmillos',
 'es un orco',
 'es un ser humano',
 'es nordico',
 'es un imperial',
 'es un breton',
 'usa magia de forma natural',
 'es un alto elfo',
 'es muy habil con las armas pesadas',
 'es muy habil al ser sigiloso']

## Reemplaza los atomos con sus numeros en la regla

In [75]:
reglasx = []
NoPrep : str

for regla in l_reglas:
    temp = regla
    for index, atomo in enumerate(atomos_unicos):
        NoPrep = "P" + str(index)
        if atomo in temp:
            temp.replace(atomo, NoPrep)
            reglasx.append(temp)
            temp = ""
            NoPrep = ""

reglasx


['~ tiene orejas puntiagudas -> ~ es un elfo oscuro V ~ es un elfo alto V ~ es un elfo del bosque',
 'tiene pelaje V sus ojos tienen pupilas verticales -> es un kahjiit V es un argoniano',
 '~ es un elfo alto -> es un elfo oscuro V es un elfo del bosque',
 'tiene piel oscura -> es un elfo oscuro V es un guarda rojo',
 'tiene piel clara & ~ tiene orejas puntiagudas -> es un nordico',
 'tiene colmillos -> es un orco V es un kahjiit',
 'es un ser humano & ~ es un nordico V ~ es un guarda rojo -> es un imperial V es un breton',
 'usa magia de forma natural -> es un breton V es un elfo oscuro V es un alto elfo',
 'es muy habil con las armas pesadas -> es un orco V es un nordico V es un guarda rojo',
 'es muy habil al ser sigiloso -> es un kahjiit V es un elfo del bosque']

## Arbol binario

In [44]:
class Arbol:
    # Funciones privadas
    def __init__(self, dato):
        self.raiz = Nodo(dato)

    def __agregar_recursivo(self, nodo, dato):
        if dato < nodo.dato:
            if nodo.izquierda is None:
                nodo.izquierda = Nodo(dato)
            else:
                self.__agregar_recursivo(nodo.izquierda, dato)
        else:
            if nodo.derecha is None:
                nodo.derecha = Nodo(dato)
            else:
                self.__agregar_recursivo(nodo.derecha, dato)

    def __inorden_recursivo(self, nodo):
        if nodo is not None:
            self.__inorden_recursivo(nodo.izquierda)
            print(nodo.dato, end=", ")
            self.__inorden_recursivo(nodo.derecha)

    def __preorden_recursivo(self, nodo):
        if nodo is not None:
            print(nodo.dato, end=", ")
            self.__preorden_recursivo(nodo.izquierda)
            self.__preorden_recursivo(nodo.derecha)

    def __postorden_recursivo(self, nodo):
        if nodo is not None:
            self.__postorden_recursivo(nodo.izquierda)
            self.__postorden_recursivo(nodo.derecha)
            print(nodo.dato, end=", ")

    def __buscar(self, nodo, busqueda):
        if nodo is None:
            return None
        if nodo.dato == busqueda:
            return nodo
        if busqueda < nodo.dato:
            return self.__buscar(nodo.izquierda, busqueda)
        else:
            return self.__buscar(nodo.derecha, busqueda)

    # Funciones públicas

    def agregar(self, dato):
        self.__agregar_recursivo(self.raiz, dato)

    def inorden(self):
        print("Imprimiendo árbol inorden: ")
        self.__inorden_recursivo(self.raiz)
        print("")

    def preorden(self):
        print("Imprimiendo árbol preorden: ")
        self.__preorden_recursivo(self.raiz)
        print("")

    def postorden(self):
        print("Imprimiendo árbol postorden: ")
        self.__postorden_recursivo(self.raiz)
        print("")

    def buscar(self, busqueda):
        return self.__buscar(self.raiz, busqueda)

## Motor de inferencia